# MNAD Evaluation

## Libraries import

In [1]:
import argparse
import os
import torch
import torch.utils.data as data
import torchvision.transforms as T
import torch.nn as nn

from data.CustomDataset import CustomImageDataset
from evaluation_utils import *

## Constants

In [2]:
DATASET_DIR_SUFFIX = 'images'

## Parameters

In [3]:
args_dict = {
  "gpus": "1",                            # gpus (set 1 or None)
  "batch_size": 1,                        # batch size for testing
  "h": 256,                               # height of input images
  "w": 256,                               # width of input images
  "c": 3,                                 # channel of input images
  "method": "recon",                      # The target task for anoamly detection  (pred or recon)
  "t_length": 1,                          # length of the frame sequences
  "fdim": 512,                            # channel dimension of the features
  "mdim": 512,                            # channel dimension of the memory items
  "msize": 10,                            # number of the memory items
  "alpha": 0.7,                           # weight for the anomality score
  "th": 0.015,                            # threshold for test updating
  "num_workers": 1,                       # number of workers for the test loader
  "dataset_type": "clean_road",           # type of dataset: clean_road
  "dataset_path": "./dataset",            # directory of data
  "label_path": "./dataset",              # directory of labels
  "label_file": "metadata.csv",           # name of the label file
  "model_path": "model/trained",          # directory of model
  "model_file": "model.pth",              # name of the model file
  "m_items_path": "model/trained",        # directory of memory items
  "m_items_file": "keys.pt"               # name of the memory items file
}

args = argparse.Namespace(**args_dict)

## GPU Configurations

In [4]:
print(torch.cuda.is_available())
if args.gpus is not None and torch.cuda.is_available():
  print(torch.cuda.is_available())

  print(torch.cuda.device_count())

  print(torch.cuda.current_device())

  print(torch.cuda.device(0))

  print(torch.cuda.get_device_name(0))

True
True
1
0
Quadro T2000 with Max-Q Design


In [5]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
if args.gpus is None:
    gpus = "0"
    os.environ["CUDA_VISIBLE_DEVICES"]= gpus
else:
    gpus = ""
    for i in range(len(args.gpus)):
        gpus = gpus + args.gpus[i] + ","
    os.environ["CUDA_VISIBLE_DEVICES"]= gpus[:-1]

#torch.backends.cudnn.enabled = True # make sure to use cudnn for computational performance

## Data loading

In [6]:
test_folder = os.path.join(args.dataset_path, args.dataset_type, 'testing', DATASET_DIR_SUFFIX)
test_label_file = os.path.join(args.label_path, args.dataset_type, args.label_file)

#transform = T.Resize((args.h,args.w))
transform = T.Compose([T.ToTensor(),])

# Loading dataset
test_dataset = CustomImageDataset(test_label_file, test_folder, transform = transform, use_cv2=True)
test_size = len(test_dataset)

test_batch = data.DataLoader(test_dataset, batch_size = args.batch_size,
                              shuffle=True, num_workers=args.num_workers, drop_last=False)
batch_size = len(test_batch)

## Model loading

In [7]:
# Loading the trained model
model_file = os.path.join(args.model_path, args.dataset_type, args.model_file)
m_items_file = os.path.join(args.m_items_path, args.dataset_type, args.m_items_file)
#model = torch.load(args.model_dir, map_location=torch.device('cpu'))
model = torch.load(model_file)
model.cuda()
#m_items = torch.load(args.m_items_dir, map_location=torch.device('cpu'))
m_items = torch.load(m_items_file)

## Evaluation

In [8]:
loss_func_mse = nn.MSELoss(reduction='none')

psnr_list = {}
feature_distance_list = {}

# Populate the dictionaries with the image names and empty lists
for img_name in test_dataset.imgs_labels["filename"].to_numpy():
    psnr_list[img_name] = []
    feature_distance_list[img_name] = []

m_items_test = m_items.clone()

In [9]:
print('Evaluation of', args.dataset_type)
model.eval()

for j,(images, labels) in enumerate(test_batch):
    imgs = images["file"]
    img_name = images["name"][0]

    if args.gpus is not None and torch.cuda.is_available():
        imgs = imgs.cuda()

    if args.method == 'pred':
        outputs, feas, updated_feas, m_items_test, softmax_score_query, softmax_score_memory, _, _, _, compactness_loss = model.forward(imgs[:,0:3*4], m_items_test, False)
        mse_imgs = torch.mean(loss_func_mse((outputs[0]+1)/2, (imgs[0,3*4:]+1)/2)).item()
        mse_feas = compactness_loss.item()

        # Calculating the threshold for updating at the test time
        point_sc = point_score(outputs, imgs[:,3*4:])

    else:
        outputs, feas, updated_feas, m_items_test, softmax_score_query, softmax_score_memory, compactness_loss = model.forward(imgs, m_items_test, False)
        mse_imgs = torch.mean(loss_func_mse((outputs[0]+1)/2, (imgs[0]+1)/2)).item()
        mse_feas = compactness_loss.item()

        # Calculating the threshold for updating at the test time
        point_sc = point_score(outputs, imgs)

    if  point_sc < args.th:
        query = nn.functional.normalize(feas, dim=1)
        query = query.permute(0,2,3,1) # b X h X w X d
        m_items_test = model.memory.update(query, m_items_test, False)
    
    psnr_list[img_name].append(psnr(mse_imgs))
    feature_distance_list[img_name].append(mse_feas)

# TODO: Check this part
# Measuring the abnormality score and the AUC
anomaly_score_total_list = []
# I removed the normalization of psnr_list and feature_distance_list done over whole video frames but maybe I need to normalize.
# The normalization was performed by the functions anomaly_score_list and anomaly_score_list_inv.
# Old code:
#for video in sorted(videos_list):
    #video_name = video.split('/')[-1]
    #anomaly_score_total_list += score_sum(anomaly_score_list(psnr_list[video_name]), 
                                     #anomaly_score_list_inv(feature_distance_list[video_name]), args.alpha)
# New code:
#for img_name in test_dataset.imgs_labels["filename"].to_numpy():
    #anomaly_score_total_list += score_sum(psnr_list[img_name], feature_distance_list[img_name], args.alpha)

# New code with single list and normalization TODO: Test result
psnr_listed = anomaly_score_list(list(psnr_list.values()))
feature_distance_listed = anomaly_score_list_inv(list(feature_distance_list.values()))
anomaly_score_total_list = score_sum(psnr_listed, feature_distance_listed, args.alpha)

anomaly_score_total_list = np.asarray(anomaly_score_total_list)

print(anomaly_score_total_list.shape)
print(anomaly_score_total_list)

# TODO: Check the correct values for labels and scores
anomaly_score_total_list = np.expand_dims(anomaly_score_total_list, axis=0)
labels_list = np.expand_dims(1 - test_dataset.imgs_labels["label"].to_numpy(), axis=0)
accuracy = AUC(anomaly_score_total_list, labels_list)

print('The result of ', args.dataset_type)
print('AUC: ', accuracy*100, '%')

Evaluation of clean_road
(129, 1)
[[0.79564784]
 [0.47154563]
 [0.41300611]
 [0.40272919]
 [0.62047204]
 [0.782453  ]
 [0.49510114]
 [0.51275119]
 [0.71267989]
 [0.20168738]
 [0.51414479]
 [0.4106049 ]
 [0.66121247]
 [0.64233846]
 [0.5082961 ]
 [0.32736858]
 [0.68735506]
 [0.65859071]
 [0.55345455]
 [0.38701058]
 [0.75586877]
 [0.44219017]
 [0.40581202]
 [0.38978965]
 [0.64609395]
 [0.62352912]
 [0.56023779]
 [0.50647671]
 [0.44092348]
 [0.56959701]
 [0.25934105]
 [0.47910843]
 [0.47461326]
 [0.41798061]
 [0.61668036]
 [0.86177   ]
 [0.66951401]
 [0.43435714]
 [0.49907334]
 [0.32850573]
 [0.54373238]
 [0.56511702]
 [0.2883019 ]
 [0.50187352]
 [0.63411324]
 [0.80725245]
 [0.27641902]
 [0.73373171]
 [0.59937038]
 [0.37326901]
 [0.2546355 ]
 [0.58826723]
 [0.21599441]
 [0.68931818]
 [0.63229006]
 [0.5739847 ]
 [0.58092392]
 [0.58567944]
 [0.55630591]
 [0.67396715]
 [0.58806113]
 [0.36376716]
 [0.30218809]
 [0.65400275]
 [0.57723189]
 [0.54373238]
 [0.5217744 ]
 [0.55644882]
 [0.62884397]
